<a href="https://colab.research.google.com/github/TimKo90/crucible-analysis-notebooks/blob/main/insitu-analysis/InSituPL_Fitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from matplotlib import ticker
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
from scipy import signal
import scipy.integrate as integrate
from scipy.interpolate import interp1d
import copy
import os
import traceback

In [ ]:
# ================Functions=================================================

def sum_of_Voigts(x, *params):

    if isinstance(x, float):
        x = np.array([x])

    params = np.array(params)
    n = (len(params)-2) // 4

    # divide parameters
    amps   = params[:n]
    mus    = params[n:2*n]
    sigmas = params[2*n:3*n]
    alphas = params[3*n:4*n]

    gaussians  = amps*np.exp(-(x[:, np.newaxis] - mus)**2 / sigmas)
    lorentian  = np.log(2) * (2/np.pi)**0.5 * (amps*sigmas / ((x[:, np.newaxis] - mus)**2 + sigmas*np.log(2)))
    background = params[-2]*x + params[-1]

    return np.dot(gaussians, 1-alphas) + np.dot(lorentian, alphas) + background


def background(x, y0, y1):
    return y0*x + y1


def fWHM_Voigt(x, center, maxValue, params):

    x1 = np.linspace(x[0], center, 5001)
    x2 = np.linspace(center, x[-1], 5001)

    y1 = sum_of_Voigts(x1, *params)
    y2 = sum_of_Voigts(x2, *params)

    root1 = np.interp(maxValue/2,y1,x1)
    root2 = np.interp(maxValue/2,y2[::-1],x2[::-1])

    return root2 - root1


def find_zero_crossings(dy, threshold):
    """
    Find zero crossings with a threshold.

    Args:
    dy: numpy array of y-values (typically the first derivative)
    threshold: tolerance around zero (default is 1e-6)

    Returns:
    numpy array of indices where zero crossings occur
    """
    zero_crossings = []
    for i in range(1, len(dy)):
        if (dy[i-1] < -threshold and dy[i] > threshold) or \
           (dy[i-1] > threshold and dy[i] < -threshold) or \
           (abs(dy[i-1]) <= threshold and abs(dy[i]) <= threshold):
            zero_crossings.append(i)
    return np.array(zero_crossings)

def peakFinder(x, y, t, num, peak_height_threshold):
    smoothing_window=11
    poly_order=5

    # Assuming x and y are your original data points
    x_original = x
    y_original = y  # Your original y values

    # Create an interpolation function
    f = interp1d(x_original, y_original, kind='cubic')

    # Create new x values for 1000 points
    x_new = np.linspace(np.min(x), np.max(x), 1000)

    # Interpolate y values
    y_interpolated = f(x_new)

    # Apply Savitzky-Golay filter to the interpolated data
    y_smooth = savgol_filter(y_interpolated, smoothing_window, poly_order)

    # Calculate first and second derivatives
    dy = np.gradient(y_smooth)
    d2y = np.gradient(dy)

    # Usage
    zero_crossings = find_zero_crossings(dy, threshold=1e-4)

    # Check if second derivative is negative at these points (maxima)
    peak_candidates = zero_crossings[d2y[zero_crossings] < 0]

    # Filter peak candidates based on height
    high_peaks = peak_candidates[y_smooth[peak_candidates] > peak_height_threshold]

    return x_new[high_peaks]


def plFitting(inputDict, df_yCut, df_xCutFit, df_fit, show_every, numGauss, peakLowerTH, peakUpperTH, estPeakWidth, minPeakWidth, maxPeakWidth, name_d, name, PLFits_CenterGuesses, PLFits_Propagate):

    estPositions = PLFits_CenterGuesses

    frames = range(0, len(df_xCutFit))
    frames_to_plot = [i for i in frames if i % show_every == 0]

    yVals = np.copy(df_fit)
    popt = np.array([[np.nan, np.nan, np.nan, np.nan]*int(numGauss) + [np.nan, np.nan]] * np.shape(df_fit)[1])
    peakFWHM = np.array([[np.nan]*int(numGauss)] * np.shape(df_fit)[1])
    peakArea = np.array([[np.nan]*int(numGauss)] * np.shape(df_fit)[1])

    # The next block is to convert the estimated peak positions and ranges into indexes
    idxLowerTH = [0.0]*int(numGauss)
    idxUpperTH = [0.0]*int(numGauss)

    for i in range(0, int(numGauss)):
        idxLowerTH[i] = next(xStart for xStart, valStart in enumerate(df_yCut) if valStart > peakLowerTH[i])
        idxUpperTH[i] = next(xEnd for xEnd, valEnd in enumerate(df_yCut) if valEnd > peakUpperTH[i])

    firstSpectrum = True
    for i in range(0, np.shape(df_fit)[1]):

        # get y values
        yVals[:, i] = np.where(yVals[:, i] == float('inf'), 5, yVals[:, i])

        idx = np.argmax(yVals[0:idxUpperTH[0], i])
        yVals[idx, i] = yVals[idx - 1, i]

        # find peaks
        peak_height_threshold = np.percentile(df_cut, 50)
        xPeaks = peakFinder(df_yCut, yVals[:, i], df_xCutFit, i, peak_height_threshold)

        # no peak, skip
        if len(xPeaks) == 0:
            print(f"No peak found at t = {df_xCutFit[i]} s.")
            continue

        peaks = np.empty(len(xPeaks), dtype=int)
        for ii in range(0, len(xPeaks)):
            peaks[ii] = int(np.abs(df_yCut - xPeaks[ii]).argmin())
        numPeaks = len(peaks)

        # array initialization
        estAmplitudes = [0.0]*int(numGauss)
        minAmplitudes = [0.0]*int(numGauss)
        maxAmplitudes = [0.0]*int(numGauss)
        estAlphas = [0.5]*int(numGauss)
        minAlphas = [0.0]*int(numGauss)
        maxAlphas = [1.0]*int(numGauss)
        minLinBkg = -1000.0
        estLinBkg = yVals[-1,i] - yVals[0,i]
        maxLinBkg = 1000.0
        minConstBkg = np.min(yVals[:,i])-100
        estConstBkg = np.min(yVals[:,i])
        maxConstBkg = np.max(yVals[:,i])

        if firstSpectrum:
            firstSpectrum = False
            firstFitIdx = i

            # find initial parameters and bounds for peak amplitudes, having free peaks start more prominent than propagating ones
            for ii in range(0,int(numGauss)):
                if float(PLFits_Propagate[ii]):
                    estAmplitudes[ii] = max(yVals[idxLowerTH[ii]:idxUpperTH[ii], i]) / 5
                    minAmplitudes[ii] = 0
                    maxAmplitudes[ii] = max(yVals[idxLowerTH[ii]:idxUpperTH[ii], i]) / 1.5
                else:
                    estAmplitudes[ii] = max(yVals[idxLowerTH[ii]:idxUpperTH[ii], i])
                    minAmplitudes[ii] = estAmplitudes[ii] / 10
                    maxAmplitudes[ii] = np.inf

            # collecting fit parameters
            estParams = estAmplitudes + estPositions + estPeakWidth + estAlphas + [estLinBkg, estConstBkg]
            lowerBounds = minAmplitudes + peakLowerTH + minPeakWidth + minAlphas + [minLinBkg, minConstBkg]
            upperBounds = maxAmplitudes + peakUpperTH + maxPeakWidth + maxAlphas + [maxLinBkg, maxConstBkg]

        else:
            # update initial parameters and bounds. Propagating peaks have their position and width linked to the first one
            for ii in range(0,int(numGauss)):

                if float(PLFits_Propagate[ii]):
                    estAmplitudes[ii] = max(yVals[idxLowerTH[ii]:idxUpperTH[ii], i]) / 10
                    minAmplitudes[ii] = 0
                    maxAmplitudes[ii] = np.inf
                    estPositions[ii] = popt[firstFitIdx,int(numGauss)+ii]
                    peakLowerTH[ii] = estPositions[ii] * 0.999
                    peakUpperTH[ii] = estPositions[ii] * 1.001
                    estPeakWidth[ii] = popt[firstFitIdx][2*int(numGauss)+ii]
                    minPeakWidth[ii] = estPeakWidth[ii] / 1.01
                    maxPeakWidth[ii] = estPeakWidth[ii] * 1.01

                else:
                    estAmplitudes[ii] = max(yVals[idxLowerTH[ii]:idxUpperTH[ii], i])
                    minAmplitudes[ii] = 0
                    maxAmplitudes[ii] = np.inf

            # collecting fit parameters
            estParams   = estAmplitudes + estPositions + estPeakWidth + estAlphas + [estLinBkg, estConstBkg]
            lowerBounds = minAmplitudes + peakLowerTH + minPeakWidth + minAlphas + [minLinBkg, minConstBkg]
            upperBounds = maxAmplitudes + peakUpperTH + maxPeakWidth + maxAlphas + [maxLinBkg, maxConstBkg]

        # try fitting
        try:
            popt[i], pcov = curve_fit(sum_of_Voigts,
                                   df_yCut,
                                   yVals[:, i],
                                   p0     = estParams,
                                   bounds = (lowerBounds, upperBounds)
                                   )


        except Exception:
            print(f"The spectrum at t = {df_xCutFit[i]} s was not fitted.")
            # traceback.print_exc()
            pass

        for ii in range(0,int(numGauss)):
            parameters = [popt[i,ii], popt[i,int(numGauss)+ii], popt[i,2*int(numGauss)+ii], popt[i,3*int(numGauss)+ii], 0, 0]
            peakFWHM[i,ii] = fWHM_Voigt(df_yCut, popt[i,int(numGauss)+ii], sum_of_Voigts(popt[i,int(numGauss)+ii], *parameters), parameters)
            peakArea[i,ii] = integrate.quad(lambda x: sum_of_Voigts(x, *parameters), -np.inf,np.inf)[0]

        # plotting fit results for pre-selected frames
        if i in frames_to_plot:

            plt.figure(figsize=(6, 5))
            plt.plot(df_yCut, yVals[:, i], 'o', label='data')
            plt.plot(df_yCut, sum_of_Voigts(df_yCut, *popt[i,:]), 'r-', label='fit')

            for ii in range(0, int(numGauss)):
                plt.plot(df_yCut, sum_of_Voigts(df_yCut, *[popt[i,ii], popt[i,int(numGauss)+ii], popt[i,2*int(numGauss)+ii], popt[i,3*int(numGauss)+ii], 0, 0]), '--', label='Peak ' + str(ii+1))

            plt.plot(df_yCut, background(df_yCut, *[popt[i,-2], popt[i,-1]]), 'k--', label='Background')
            plt.legend()
            plt.xlabel('Energy (eV)')
            plt.ylabel('Intensity (a.u.)')
            plt.title('Time: ' + str(df_xCutFit[i]))
            plt.savefig(os.path.join(name + '/fits/', str(name_d) + '_PL-fit_' + str(int(df_xCutFit[i])) + '_s.png'), format = 'png')
            plt.show(block=False)
            plt.pause(1)

            if inputDict['logplots']:
                plt.figure(figsize=(6, 5))
                plt.plot(df_yCut, np.log(yVals[:, i]), 'o', label='data')
                plt.plot(df_yCut, np.log(sum_of_Voigts(df_yCut, *popt[i,:])), 'r-', label='fit')
                plt.legend()
                plt.xlabel('Energy (eV)')
                plt.ylabel('Log-Intensity (a.u.)')
                plt.title('Time: ' + str(df_xCutFit[i]))
                plt.savefig(os.path.join(name + '/fits/', str(name_d) + '_PL-fit_Log_' + str(int(df_xCutFit[i])) + '_s.png'), format = 'png')
                plt.show(block=False)
                plt.pause(1)

    # Plotting the time-evolution of the peak-positions and intensities
    for i in range(0, int(numGauss)):
        fig, ax1 = plt.subplots(figsize=(6, 5))
        plot1, = ax1.plot(df_xCutFit, popt[:,int(numGauss)+i], label = 'Peak Position')
        ax2 = ax1.twinx()
        plot2, = ax2.plot(df_xCutFit, popt[:,i], 'g', label = 'Peak Intensity')
        ax1.set_xlabel('Time (s)')
        ax1.set_ylabel(r'PL Position (eV)')
        ax2.set_ylabel(r'PL Intensity (a.u.)')
        # Create your ticker object with M ticks
        yticks = ticker.MaxNLocator(5)
        ax1.yaxis.set_major_locator(yticks)
        fig.suptitle('Fit Results Peak ' + str(i+1) + ' ' + name_d, fontsize=14)
        fig.legend()

    # collecting the fit results in a dataframe
    dfPeaks = pd.DataFrame()
    dfPeaks['Fit-Time_' + name_d] = df_xCutFit
    for i in range(0,int(numGauss)):
        colPos = 'Peak' + str(i+1) + 'Pos_' + name_d
        colArea = 'Peak' + str(i+1) + 'Area_' + name_d
        colFWHM = 'Peak' + str(i+1) + 'FWHM_' + name_d
        colAlphas = 'Peak' + str(i+1) + 'Alpha_' + name_d
        data = np.array([peakArea[:,i], popt[:,int(numGauss)+i], peakFWHM[:,i], popt[:,3*int(numGauss)+i]])
        dfTemp = pd.DataFrame(
            data.T,
            columns=[colArea, colPos, colFWHM, colAlphas])
        dfPeaks = pd.concat([dfPeaks, dfTemp], axis=1)
    dfPeaks = dfPeaks.fillna('nan')

    return dfPeaks


def plFits(inputDict, energyPL, timePL, intPL, sampleName, savePath, numGauss,
           PLFits_CenterGuesses = ["list of length num gauss containing the user's Initial guess for Peak position (in eV) as a float"],
           PLFits_CenterFixed = ["list of True/False will the center be fixed"],
           PLFits_Propagate=['list of True/False for propagate']):

    # Fit-parameters: Set the lower and upper limits as well as the estimated position of each peak (in nm).
    # From left to right, update as many integers as needed but keep the length of arrays at 5; extra values are ignored.
    peakLowerTH = [0.0]*int(numGauss)
    peakUpperTH = [0.0]*int(numGauss)
    estPeakWidth = [0.0] *int(numGauss)
    minPeakWidth = [0.0] *int(numGauss)
    maxPeakWidth = [0.0] *int(numGauss)


    for i in range(0, int(numGauss)):

        if PLFits_CenterFixed[i] is True:
            peakLowerTH[i] = float(PLFits_CenterGuesses[i]) - 0.005
            peakUpperTH[i] = float(PLFits_CenterGuesses[i]) + 0.005
            estPeakWidth[i] = (0.1/1.665)**2
            minPeakWidth[i] = (0.0/1.665)**2
            maxPeakWidth[i] = (0.5/1.665)**2
        else:
            peakLowerTH[i] = energyPL[0]
            peakUpperTH[i] = energyPL[-2]
            estPeakWidth[i] = (0.1/1.665)**2
            minPeakWidth[i] = (0/1.665)**2
            maxPeakWidth[i] = (1/1.665)**2

    show_every = int(len(timePL)/10)     # shows every tenth frame with fit

    df_fitResults = plFitting(inputDict, energyPL, timePL, intPL, show_every, numGauss, peakLowerTH, peakUpperTH, estPeakWidth, minPeakWidth, maxPeakWidth, sampleName, savePath, PLFits_CenterGuesses, PLFits_Propagate)

    return df_fitResults



In [ ]:
# mount your google drives
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
inputDict = {
               'smoothing' : False,   # Smoothing of the data to reduce noise
               'sFactor' : 3,         # Parameter for smoothing with a SavGol-Filter
               'energyScale' : True,
               'bkgCorr' : False,     # Enable linear background removal. If True, the program will ask for two ranges for the removal. I recommend setting one of them at higher and the other at lower energy compared to the peaks of interest.
               'bkgCorrPoly' : 1,     # This parameter determines the order of the polynomial fit used for background correction (0=const, 1=linear, etc.)
               'logplots': 0,
               }

### Load and View Sample

In [ ]:
project_id = "MFP08449"
sample_options = glob.glob(f"/content/drive/Shareddrives/{project_id}/Datasets/*/*_dataframe.csv")
print("Samples found at the path above are: ")
for s in sample_options:
  print(s)

In [ ]:
sample_path = "" # copy path from output above
sample_name = os.path.basename(sample_path).split("_dataframe.csv")[0]

In [ ]:
df = pd.read_csv(f"{sample_path}", index_col = "wl")

num_sort = [str(x) for x in sorted([float(x) for x in df.columns])]
df = df.reindex(num_sort, axis=1)

# load and convert to arrays
df_y = np.array(df.index)
df_x = np.array([float(x) for x in df.columns])

df = df.to_numpy()
df = np.where(df <= 0, 0, df)

# transition to energy scale of the y axis
if inputDict['energyScale'] == True:
    df_y = [1240 / i for i in df_y]

    # Jacobian transformation for all measured PL values (basically dividing by E^2)
    for i in range(np.shape(df)[1]):
        df[:, i] = df[:, i] / df_y / df_y

    # Mirroring dataframes to prevent sorting issues
    df_y = np.flip(df_y)
    df = np.flip(df, axis=0)

    yAxisLabel = 'Energy (eV)'

else:
    yAxisLabel = 'Wavelength (nm)'

# Simple plot of the data
plot_name = f"{sample_name}_2D_Plot.png"

plt.figure(1)
plt.contourf(df_x, df_y, df, 20, cmap=plt.cm.jet)

# Make a colorbar for the ContourSet
cbar = plt.colorbar()
cbar.ax.set_ylabel('Intensity (a.u.)')

# adding labels
plt.xlabel('Time (s)')
plt.ylabel(yAxisLabel)
plt.title(plot_name.strip(".png"))

plt.savefig(plot_name)

### (Optional) Run Background Subtraction

In [ ]:
# define the following in eV or nm ( you can pick a range on either side of the peak )

# background correction range 1 start/end value
y_bkgStart1 =0
y_bkgEnd1 = 1

# background correction range 2 start/end value
y_bkgStart2 = 3
y_bkgEnd2 = 5

In [ ]:
# run bkgd subtraction
l_bkgStart1 = next(xStart for xStart, valStart in enumerate(df_y) if valStart > y_bkgStart1)
l_bkgEnd1 = next(xEnd for xEnd, valEnd in enumerate(df_y) if valEnd > y_bkgEnd1) - 1
l_bkgStart2 = next(xStart for xStart, valStart in enumerate(df_y) if valStart > y_bkgStart2)
l_bkgEnd2 = next(xEnd for xEnd, valEnd in enumerate(df_y) if valEnd > y_bkgEnd2) - 1

for i in range(0, len(df_x)):
    xVals = np.concatenate([df_y[l_bkgStart1:l_bkgEnd1], df_y[l_bkgStart2:l_bkgEnd2]])
    yVals = np.concatenate([df[l_bkgStart1:l_bkgEnd1, i], df[l_bkgStart2:l_bkgEnd2, i]])
    coefs = np.polyfit(xVals, yVals, 1)  # The last parameter determines the order of the polynomial fit (0=const)
    poly1d_fn = np.poly1d(coefs)
    df[:, i] = df[:, i] - poly1d_fn(df_y[:])

### Plot Region of Interest

In [ ]:
# define the following
startTime = 0.0
endTime = 90.0

# Wavelength/Energy region to be plotted
y_start = 1.4
y_end = 1.9

# Intensity range (replace 'default' with custom range if needed)
z_start = 'default'
z_end = 'default'

In [ ]:
# run
startTime = max(0, startTime)
endTime = min(endTime, df_x[-2])
y_start = (max(df_y[0], y_start))
y_end = min(y_end, df_y[-2])

startTimeIdx = next(tStart for tStart, valStart in enumerate(df_x) if valStart > startTime) - 1
endTimeIdx = next(tStart for tStart, valStart in enumerate(df_x) if valStart > endTime) + 1
l_start = next(xStart for xStart, valStart in enumerate(df_y) if valStart > y_start) - 1
l_end = next(xEnd for xEnd, valEnd in enumerate(df_y) if valEnd > y_end) + 1

# shortening the wavelength/energy region to interesting region
df_cut = df[l_start:l_end, startTimeIdx:endTimeIdx]
df_cutFit = df[:, startTimeIdx:endTimeIdx]

# same shortening for the Y-axis and X-axis arrays
df_yCut = df_y[l_start:l_end]
df_xCut = df_x[startTimeIdx:endTimeIdx]
df_xCut = df_xCut - startTime

if inputDict['smoothing']:
    df_cut = savgol_filter(df_cut, inputDict['sFactor'], 0)
    df_cutFit = savgol_filter(df_cutFit, inputDict['sFactor'], 0)

if z_start == 'default':
    z_start = np.min(df_cut)
if z_end == 'default':
    z_end = np.max(df_cut)

# Show the resulting data
plt.figure(3)
plt.contourf(df_xCut, df_yCut, df_cut, 20, cmap=plt.cm.jet, levels = np.linspace(z_start,z_end,100))
# Make a colorbar for the ContourSet
cbar = plt.colorbar()
cbar.ax.set_ylabel('Intensity (a.u.)')
cbar.set_ticks(np.arange(z_start,z_end+1,(z_end-z_start)/10))
# adding labels
plt.xlabel('Time (s)')
plt.ylabel(yAxisLabel)
plt.title(str(sample_name) + ' _2D_Plot')
plt.pause(1)

### Bin the data
sum a certain number (bin_size) of spectra to improve the fitting accuracy at the cost of time resolution

In [ ]:
# define the number of spectra to sum per bin
# set to 0 if you don't want to bin
bin_size = 0

In [ ]:
if bin_size > 0:
    df_cutBin = copy.deepcopy(df_cutFit)
    df_cutBin = df_cutFit[:, 0:int(df_xCut.shape[0]/bin_size)]
    for i in range(0, df_cutBin.shape[1]):
        df_cutBin[:,i] = np.sum(df_cutFit[:, bin_size*i:bin_size*i+bin_size], axis=1)
    df_fit = df_cutBin
    df_xCutFit = df_xCut[::inputDict['binning']][0:df_cutBin.shape[1]]
    df_yCut = df_y
else:
    df_fit = df_cut
    df_xCutFit = df_xCut

### Fit the Data

In [ ]:
# set the following parameters
numGauss = 2
PLFits_CenterGuesses = [1.61, 1.70] # for each gaussian you think exists, put a float of the eV/wl at which you think there is a peak
PLFits_CenterFixed = [False, False] # for each gaussian, put a value True/False if you think the center is fixed
PLFits_Propagate = [False, False] # for each gaussian, put a value True/False for propagate

In [ ]:
# you can rerun this to restart the fit
result_path = os.path.join(os.path.dirname(sample_path), "ProcessedData")
try:
  os.mkdir(result_path)
except Exception as e:
  print(e)
df_fitResults = plFits(inputDict, df_yCut, df_xCutFit, df_fit, sample_name, os.path.dirname(sample_path), numGauss, PLFits_CenterGuesses, PLFits_CenterFixed, PLFits_Propagate)

### Select individual spectra to plot

In [ ]:
# Run this if you want to extract individual points in time
# Add the times (in s) that you'd like to extract to this list
spectra = [0, 5, 10, 90]


idxToPlot = []
for i in range(0,len(spectra)):
        idx = next(tStart for tStart, valStart in enumerate(df_xCutFit) if valStart > float(spectra[i]))
        idxToPlot.append(idx)

In [ ]:
# Run this if you want to extract all spectra in a specific time range
# Add the times (in s) that you'd like to use as range
firstSpectrum = 0
lastSpectrum = 50


idxToPlot = []
startSpecIdx = next(tStart for tStart, valStart in enumerate(df_xCutFit) if valStart > float(firstSpectrum))
endSpecIdx = next(tEnd for tEnd, valStart in enumerate(df_xCutFit) if valStart > float(lastSpectrum))
idxToPlot = range(startTimeIdx, endTimeIdx)

In [ ]:
intensityToPlot = []
fileName = 'Indv_PL-Spectra'
timesToPlot = [df_xCutFit[x] for x in idxToPlot]
names = ['{:.2f}'.format(x) for x in timesToPlot]
names = [sample_name + str(x) + '_s' for x in names]

for i in range(0,len(idxToPlot)):
        intensity_tmp = df_fit[:,idxToPlot[i]]
        intensityToPlot.append(intensity_tmp)

intensityToPlot_array = np.array(intensityToPlot).T

dfIndv = pd.DataFrame(intensityToPlot_array, columns=names)
dfIndv[sample_name + '_Energy_Spectra'] = df_yCut


fig = plt.figure(figsize=(7, 5))

if len(timesToPlot) < 10:
    for i in range(0,len(timesToPlot)):
        plt.plot(df_yCut, intensityToPlot[i], label = ('{:.2f}'.format((df_xCutFit[idxToPlot[i]])) + ' s'))
        plt.legend(loc="upper right")
else:
    norm = plt.Normalize(df_xCutFit[idxToPlot[0]], df_xCutFit[idxToPlot[-1]])
    cmap = plt.colormaps.get_cmap('coolwarm')
    fig, ax = plt.subplots()
    for i in range(len(timesToPlot)):
        ax.plot(df_yCut, intensityToPlot[i], c=cmap(norm(df_xCutFit[idxToPlot[i]])))
    cbar = fig.colorbar(plt.cm.ScalarMappable(cmap=cmap, norm=norm), label="Time (s)", ax=fig.gca())
    cbar.set_ticks(np.linspace(df_xCutFit[idxToPlot[0]], df_xCutFit[idxToPlot[-1]], 11))


# adding labels
plt.xlabel('Energy (eV)')
plt.ylabel('Intensity (a.u.)')
plt.title(str(sample_name) + ' ' + fileName)

plt.xlim([df_yCut[0], df_yCut[-1]])
plt.ylim(bottom=0)
plt.savefig(f'{result_path}/{sample_name}_{fileName}.png', dpi=300, bbox_inches="tight")
plt.show(block=False)
plt.pause(1)

In [ ]:
### Save the Results
np.savetxt(f'{result_path}/{sample_name}_2D.csv', df_fit, delimiter=",")
np.savetxt(f'{result_path}/{sample_name}_time.csv', df_xCutFit, delimiter=",", header='Time_' + sample_name, comments='')
np.savetxt(f'{result_path}/{sample_name}_energy.csv', df_yCut, delimiter=",", header='Energy_' + sample_name, comments='')
df_fitResults.to_csv(f"{result_path}/{sample_name}_PL_FitResults.csv", index=False)
try:
  dfIndv.to_csv(f'{result_path}/{sample_name}_Indv_PL-Spectra.csv', index=None)
except:
  pass